In [202]:
import pandas as pd
import json
import sklearn.svm as svm
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.neural_network import MLPClassifier

In [236]:
## Change the name to get your path
user = 'Yash'
file1 = 'AG04-01.csv'
file2 = 'GTW04-01.csv'
# file3 = 'GTA04-02.csv'
startTimeWork = '2017-09-12 12:03:00'
endTimeWork = '2017-09-12 14:03:00'
# startTimeActive = '2017-09-17 14:16:00'
# endTimeActive = '2017-09-17 16:16:00'
file4 = 'AG01-01.csv'
file5 = 'GTL01-02.csv'
startTimeLeisure = '2017-10-03 18:46:00'
endTimeLeisure = '2017-10-03 20:46:00'

In [213]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [214]:
def useable_Data(data, gt, timeStart, timeEnd):
    data['start.time'] = pd.to_datetime(data['start.time'])
    observedData = data[(data['start.time'] >= timeStart) & (data['start.time'] <= timeEnd)]
    observedData['index'] = range(len(observedData))
    gt['index'] = gt['time']
    observedData = pd.merge(observedData,gt[['index','coding']])
    observedData.sort_index(inplace=True)
    copy = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']].copy(deep=True)
    copy.loc[-1] = copy.loc[0]  # adding a row
    copy.index = copy.index + 1  # shifting index
    copy.sort_index(inplace=True)
    copy.columns = 'last.' + copy.columns
    observedData = pd.concat([observedData, copy], axis = 1)
    observedData = observedData.drop(observedData.index[len(observedData)-1])
    return observedData

In [215]:
def select_features_from_lasso(X, y, alpha):
    # fit lasso model and pass to select from model
    lasso = LassoCV().fit(X, y)
    model = SelectFromModel(lasso, prefit=True)

    # new features
    X_new = model.transform(X)
    return X.columns[model.get_support()]

In [250]:
data = pd.read_csv(load_Data(user, file1), header=0)
gt1 = pd.read_csv(load_Data(user, file2), header=0)
data2 = pd.read_csv(load_Data(user, file4), header=0)
gt2 = pd.read_csv(load_Data(user, file5), header=0)
observedData = pd.DataFrame(useable_Data(data, gt1, startTimeWork, endTimeWork))
observedData2 = pd.DataFrame(useable_Data(data2, gt2, startTimeLeisure, endTimeLeisure))
observedData['type'] = 'W-04'
observedData2['type'] = 'L-01'
observedData = pd.concat([observedData, observedData2])
observedData.head()

/Users/yashmehra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Unnamed: 0,coding,dfreq,index,last.dfreq,last.mean.ang,last.mean.vm,last.p625,last.ratio.df,last.sd.ang,last.sd.vm,mean.ang,mean.vm,p625,ratio.df,sd.ang,sd.vm,start.time,type
0,87481.0,non-sedentary,4.102564,0.0,4.102564,-17.691455,1.039847,0.0,0.0,0.0,0.0,-17.691455,1.039847,0.0,0.0,0.0,0.0,2017-09-12 12:03:00,W-04
1,87482.0,non-sedentary,4.102564,1.0,4.102564,-17.691455,1.039847,0.0,0.0,0.0,0.0,-17.691455,1.039847,0.0,0.0,0.0,0.0,2017-09-12 12:03:01,W-04
2,87483.0,non-sedentary,4.102564,2.0,4.102564,-17.691455,1.039847,0.0,0.0,0.0,0.0,-17.691455,1.039847,0.0,0.0,0.0,0.0,2017-09-12 12:03:02,W-04
3,87484.0,non-sedentary,4.102564,3.0,4.102564,-17.691455,1.039847,0.0,0.0,0.0,0.0,-17.691455,1.039847,0.0,0.0,0.0,0.0,2017-09-12 12:03:03,W-04
4,87485.0,non-sedentary,4.102564,4.0,4.102564,-17.691455,1.039847,0.0,0.0,0.0,0.0,-17.691455,1.039847,0.0,0.0,0.0,0.0,2017-09-12 12:03:04,W-04


In [251]:
len(observedData)

14462

In [249]:
observedData.columns

Index(['Unnamed: 0', 'mean.vm', 'sd.vm', 'mean.ang', 'sd.ang', 'p625', 'dfreq',
       'ratio.df', 'start.time', 'index', 'coding', 'last.mean.vm',
       'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq',
       'last.ratio.df', 'type', 'index', 'mean.vm', 'sd.vm', 'mean.ang',
       'sd.ang', 'p625', 'dfreq', 'ratio.df', 'start.time', 'coding',
       'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang',
       'last.p625', 'last.dfreq', 'last.ratio.df', 'type'],
      dtype='object')

## Build test and train Datasets

In [252]:
observedData = observedData.sample(frac = 1)

train = observedData[:10800]
test = observedData[10800:]

## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
X = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
y = observedData['coding']

trainX = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
trainY = train['coding']
testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
testY = test['coding']

## Lasso (Feature selection)

In [168]:
# trainY = (trainY == 'sedentary').astype(int)
# testY = (testY == 'sedentary').astype(int)

In [221]:
features = select_features_from_lasso(trainX, trainY, 0.001)

ValueError: could not convert string to float: 'sedentary'

In [187]:
features

Index(['sd.vm', 'mean.ang', 'sd.ang', 'p625', 'dfreq', 'last.sd.vm',
       'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq',
       'last.ratio.df'],
      dtype='object')

In [188]:
trainX = trainX[features]
testX = testX[features]

## SVM

In [253]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [256]:
clf.score(testX,testY)

0.8143091206990716

## Neural Network

In [257]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(trainX,trainY)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [258]:
np.sum(n_net.predict(testX) == testY) / len(testX)

0.8350628072091754